In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import *
import numpy as np
from sklearn.decomposition import *
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/sklearn/lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be re

In [2]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 80

In [3]:
##Define the object for the season team
class MyDf:
    def __init__(self, mydf):
        self.df = mydf 

In [4]:
def select_season (df, season):
    return df[df['season']==season]

In [5]:
def cut_out_n_m_matches(df,n,m):
    a=df['H_ROUND'].unique()[n-1]
    b=df['H_ROUND'].unique()[-m]
    
    return df[(df['H_ROUND']>a) & (df['H_ROUND']<b)]
    

In [6]:
def train_test_index(df,predicted_season):
    train_index=df[df.season!=predicted_season].index
    test_index=df[df.season==predicted_season].index
    return train_index, test_index

In [7]:
def train_test_index_only_round(df,predicted_round):
    train_index=df[df['H_ROUND']<predicted_round].index
    test_index=df[df['H_ROUND']==predicted_round].index
    return train_index, test_index

In [8]:
def split_train_test_data(df,train_index,test_index,col_train,col_pred,i):
        
    X_train=df.loc[train_index].iloc[:,col_train]
    Y_train=df.loc[train_index][col_pred]
    
    X_test=df.loc[test_index]
    Y_test=df.loc[test_index]

    X_test=X_test[X_test.H_ROUND==i].iloc[:,col_train]
    Y_test=Y_test[Y_test.H_ROUND==i][col_pred]
     
   

    return X_train, Y_train, X_test, Y_test

In [9]:
def Predict_whole_season_by_one (BIG, predseason, col_train, col_pred, method, method_name):
        
    tmp=pd.DataFrame()
    
    train_index, test_index = train_test_index(BIG,predseason)
    lis=[]
    lis_2=[]
    lis_4=[]
    
    for i in BIG[BIG.season==predseason].H_ROUND.unique():
       
        X_train,Y_train,X_test,Y_test = split_train_test_data(BIG,train_index,test_index,col_train,col_pred,i) 
        method.fit(X_train,Y_train)
        #lis_2=lis_2+list(method.predict(X_test))
        #lis_4=lis_4+list(Y_test)
        lis.append(float(method.score(X_test, Y_test)))
        #print(X_test)
        #print(method.predict_proba(X_test))
        #print(Y_test)

    tmp[method_name]=lis 
    
    #tmp_2=BIG.loc[list(test_index)].iloc[:,[1,2,3,4,8]]
    
    
    
    #tmp_2['prediction']=lis_2
    #tmp_2['real']=lis_4
    
    return tmp #,tmp_2

In [10]:
def Predict_whole_season_with_one (BIG, predseason, col_train, col_pred,method, method_name):
        
    tmp=pd.DataFrame()    
    
    train_index, test_index = train_test_index(BIG,predseason)
    lis=[]
    lis_2=[]

    
    for i in BIG[BIG.season==predseason].H_ROUND.unique():
       
        X_train,Y_train,X_test,Y_test = split_train_test_data(BIG,train_index,test_index,col_train,col_pred,i) 
        method.fit(X_train,Y_train)
        lis.append(float(method.score(X_test, Y_test)))
        
        train_index=list(train_index)+list(X_test.index)
       
    tmp[method_name]=lis     
    return tmp
    

In [11]:
def Predict_season_with_current_season (df, predseason, col_train, col_pred,method, method_name):
    
    tmp=pd.DataFrame()
    tmp_2=df[df['season']==predseason]
    lis=[]
    
    second_round=tmp_2['H_ROUND'].unique()[1]
        
    for i in range(int(df.H_ROUND.unique()[1]), int(df.H_ROUND.unique()[-1])):
        
        train_index, test_index = train_test_index_only_round(df,i)
        
        X_train=df.loc[train_index].iloc[:,col_train]
        Y_train=df.loc[train_index][col_pred]

        
        X_test=df.loc[test_index].iloc[:,col_train]
        Y_test=df.loc[test_index][col_pred]
        
       
        method.fit(X_train,Y_train)
        
        lis.append(float(method.score(X_test, Y_test)))
        
       
    tmp[method_name]=lis     
    
    return tmp


In [12]:
def find_best_model (X,col_pred,col_train,method):
    n=100
    from sklearn.cross_validation import train_test_split
    tmp=0
    for i in range(n):
        X_train, X_test, y_train, y_test = train_test_split(X.iloc[:,column_to_use_as_prediction], X[col_pred], test_size=0.20)
        method.fit(X_train,y_train)
        if method.score(X_test,y_test)>tmp:
            Best=method.fit(X_train,y_train)
    return Best

In [13]:
def pred_all_meth (df, season_to_be_predicted,column_to_predict,column_to_use_as_prediction):
    
    lis=[]
    
    #column_to_use_as_prediction = [22,18,51,49,42,20]
    #column_to_predict='Goal/No_Goal'
    
    svm_linear = svm.SVC(kernel='linear')
    lis.append(MyDf(Predict_whole_season_by_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, svm_linear,'svm_linear_1')))
    lis.append(MyDf(Predict_whole_season_with_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, svm_linear,'svm_linear_2')))
    #lis.append(MyDf(Predict_season_with_current_season(df,season_to_be_predicted, column_to_use_as_prediction, column_to_predict,svm_linear,'svm_linear_3')))    
    
    #column_to_use_as_prediction = [22,18,51,49,42,20]
    #column_to_predict='Goal/No_Goal'
    
    svm_rbf = svm.SVC(kernel='rbf')
    lis.append(MyDf(Predict_whole_season_by_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict,svm_rbf,'svm_rbf_1')))
    lis.append(MyDf(Predict_whole_season_with_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, svm_rbf,'svm_rbf_2')))
    #lis.append(MyDf(Predict_season_with_current_season(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict,svm_rbf,'svm_rbf_3')))

    
    
    #column_to_use_as_prediction = [22,18,51,49,42,20]
    #column_to_predict = 'Goal/No_Goal'  
               
    #dtc = DecisionTreeClassifier()
    #lis.append(MyDf(Predict_whole_season_by_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, dtc,'dtc_1')))
    #lis.append(MyDf(Predict_whole_season_with_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, dtc,'dtc_2')))
    #lis.append(MyDf(Predict_season_with_current_season(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, dtc,'dtc_3')))

    
    
    #column_to_use_as_prediction=[22,18,51,49,42,20]
    #column_to_predict='Goal/No_Goal'  
               
    #bagb = AdaBoostClassifier(DecisionTreeClassifier(max_depth=10), n_estimators=20)
    #lis.append(MyDf(Predict_whole_season_by_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, bagb,'bagb_1')))
    #lis.append(MyDf(Predict_whole_season_with_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, bagb,'bagb_2')))
    #lis.append(MyDf(Predict_season_with_current_season(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, bagb,'bagb_3')))          
               
    
    #column_to_use_as_prediction=[22,18,51,49,42,20]
    #column_to_predict='Goal/No_Goal'           
               
    #gradb = GradientBoostingClassifier(max_depth=10, n_estimators=20)
    #lis.append(MyDf(Predict_whole_season_by_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, gradb,'gradb_1')))
    #lis.append(MyDf(Predict_whole_season_with_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, gradb,'gradb_2')))
    #lis.append(MyDf(Predict_season_with_current_season(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, gradb,'gradb_3')))         
   
                    
    
    #column_to_use_as_prediction=[22,18,51,49,42,20]
    #column_to_predict='Goal/No_Goal'  
                    
    Ardf = AdaBoostClassifier(RandomForestClassifier(max_depth=10, n_estimators=20))
    lis.append(MyDf(Predict_whole_season_by_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, Ardf,'rdf_1')))
    lis.append(MyDf(Predict_whole_season_with_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, Ardf,'rdf_2')))
    #lis.append(MyDf(Predict_season_with_current_season(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, Ardf,'rdf_3')))
    
    
    
    return lis

In [14]:
def Put_Togheter (lis):
    tmp=pd.DataFrame()
    for i in range(len(lis)):
        tmp=tmp.join(lis[i].df, how='outer')
        
    return tmp

# Loading data

In [15]:
final=pd.read_csv('../Data/from_sqlite_Data/Realfinal_before_name_change.csv')
final.drop('Unnamed: 0',axis=1,inplace=True)
allchampionships=[]

In [16]:
percentages=pd.read_csv('../Data/1x2_all_leagues')
percentages

,Unnamed: 0,Switzerland Super League,Poland Ekstraklasa,Scotland Premier League,France Ligue 1,Germany 1. Bundesliga,England Premier League,Belgium Jupiler League,Portugal Liga ZON Sagres,Netherlands Eredivisie,Italy Serie A,Spain LIGA BBVA
0,0,0.452830,0.454812,0.429167,0.444583,0.454885,0.460899,0.467764,0.442890,0.486371,0.466056,0.493539
1,1,0.243486,0.275235,0.240972,0.288333,0.243994,0.258735,0.239369,0.254662,0.230893,0.267805,0.227178
2,2,0.303684,0.269953,0.329861,0.267083,0.301121,0.280366,0.292867,0.302448,0.282737,0.266139,0.279283


In [17]:
for i in range(7,37):
    final.iloc[:,i]=pd.to_numeric(final.iloc[:,7],errors='coerce')

In [18]:
final=final.fillna(0)

In [19]:
for i in range(len(final.country_id.unique())):
    allchampionships.append(MyDf(final[final['country_id']==final.country_id.unique()[i]].sort(columns='Date')))
    allchampionships[i].df.reset_index(inplace=True)
    allchampionships[i].df.drop(['index'],axis=1,inplace=True)

/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


# Cut out first and last n,m matches

In [20]:
n=3
m=3

In [21]:
for i in range(len(final.country_id.unique())):
    allchampionships[i].df=cut_out_n_m_matches(allchampionships[i].df,n,m)
    

In [22]:
pd.DataFrame(allchampionships[0].df.columns)

,0
0,country_id
1,season
2,Date
3,HomeTeam
4,AwayTeam
5,FTHG
6,FTAG
7,B365H
8,B365D
9,B365A


In [23]:
column_to_use_as_prediction = [90,91,92,93]

# Parameter

In [24]:
season_to_be_predicted='2015/2016'
column_to_predict='FTR'
column_to_use_as_prediction = [88,89,87,92,93]


In [25]:
svm_linear = svm.SVC(kernel='rbf')
a=Predict_season_with_current_season(allchampionships[9].df, season_to_be_predicted,column_to_use_as_prediction,column_to_predict,svm_linear,'svm')

In [26]:
a.mean()

svm    0.485473
dtype: float64

# RUN with all method only ONE LEAGUE

In [27]:
a=pred_all_meth(allchampionships[0].df, season_to_be_predicted,column_to_predict,column_to_use_as_prediction)

## Put together

In [28]:
b=Put_Togheter(a)

In [29]:
b.mean()

svm_linear_1    0.492381
svm_linear_2    0.492381
svm_rbf_1       0.473492
svm_rbf_2       0.486825
rdf_1           0.480238
rdf_2           0.443254
dtype: float64

## Get Means and Best Methods

In [30]:
for i in range(len(b.mean()[b.mean()==max(b.mean())])):
    print ('Best method are', b.mean()[b.mean()==max(b.mean())].index[i], 'with score', b.mean()[b.mean()==max(b.mean())][i])


('Best method are', 'svm_linear_1', 'with score', 0.49238095238095242)
('Best method are', 'svm_linear_2', 'with score', 0.49238095238095242)


# Run all methods ALL LEAGUES 

In [31]:
def FINAL (df,season_to_be_predicted,column_to_predict,column_to_use_as_prediction):
    lis=[]
    tmp=pd.DataFrame(columns=[u'svm_linear_1', u'svm_linear_2', u'svm_linear_3', u'svm_rbf_1',
       u'svm_rbf_2', u'svm_rbf_3', u'dtc_1', u'dtc_2', u'dtc_3', u'bagb_1',
       u'bagb_2', u'bagb_3', u'gradb_1', u'gradb_2', u'gradb_3', u'rdf_1',
       u'rdf_2', u'rdf_3'])
    
    for i in range(len(df)):
        #print(i)
        #print(allchampionships[i].df['country_id'].iloc[0])
        
        a=(pred_all_meth(df[i].df, season_to_be_predicted,column_to_predict,column_to_use_as_prediction))
        b=Put_Togheter(a)
        #print(b.mean())
        tmp=tmp.append(b.mean(), ignore_index=True)
        print(i)
    
    '''tmp.index=['Switzerland Super League', 'Poland Ekstraklasa',
       'Scotland Premier League', 'France Ligue 1',
       'Germany 1. Bundesliga', 'England Premier League',
       'Belgium Jupiler League', 'Portugal Liga ZON Sagres',
       'Netherlands Eredivisie', 'Italy Serie A', 'Spain LIGA BBVA']
    '''
    return tmp
        
        
    

In [32]:
G=FINAL(allchampionships,season_to_be_predicted,column_to_predict,column_to_use_as_prediction)

0
1
2
3
4
5
6
7
8
9
10


In [33]:
column_to_predict

'FTR'

In [108]:
G.mean()

svm_linear_1    0.479500
svm_linear_2    0.479153
svm_linear_3         NaN
svm_rbf_1       0.477707
svm_rbf_2       0.476457
svm_rbf_3            NaN
dtc_1                NaN
dtc_2                NaN
dtc_3                NaN
bagb_1               NaN
bagb_2               NaN
bagb_3               NaN
gradb_1              NaN
gradb_2              NaN
gradb_3              NaN
rdf_1           0.443195
rdf_2           0.447681
rdf_3                NaN
dtype: float64

In [39]:
season_to_be_predicted='2015/2016'
column_to_predict='Goal/No_Goal'
column_to_use_as_prediction = [90,91,92,93]

In [40]:
T=FINAL(allchampionships,season_to_be_predicted,column_to_predict,column_to_use_as_prediction)

0
1
2
3
4
5
6
7
8
9
10


In [41]:
column_to_predict


'Goal/No_Goal'

In [110]:
T.mean()

svm_linear_1    0.520684
svm_linear_2    0.518409
svm_linear_3         NaN
svm_rbf_1       0.511769
svm_rbf_2       0.509676
svm_rbf_3            NaN
dtc_1                NaN
dtc_2                NaN
dtc_3                NaN
bagb_1               NaN
bagb_2               NaN
bagb_3               NaN
gradb_1              NaN
gradb_2              NaN
gradb_3              NaN
rdf_1           0.504174
rdf_2           0.502479
rdf_3                NaN
dtype: float64

In [43]:
season_to_be_predicted='2015/2016'
column_to_predict='Over2.5/Under2.5'
column_to_use_as_prediction = [90,91,92,93]

In [44]:
R=FINAL(allchampionships,season_to_be_predicted,column_to_predict,column_to_use_as_prediction)

0
1
2
3
4
5
6
7
8
9
10


In [45]:
column_to_predict

'Over2.5/Under2.5'

In [111]:
R.mean()

svm_linear_1    0.538702
svm_linear_2    0.543317
svm_linear_3         NaN
svm_rbf_1       0.529739
svm_rbf_2       0.525934
svm_rbf_3            NaN
dtc_1                NaN
dtc_2                NaN
dtc_3                NaN
bagb_1               NaN
bagb_2               NaN
bagb_3               NaN
gradb_1              NaN
gradb_2              NaN
gradb_3              NaN
rdf_1           0.510608
rdf_2           0.514502
rdf_3                NaN
dtype: float64

In [47]:
season_to_be_predicted='2015/2016'
column_to_predict='Over1.5/Under1.5'
column_to_use_as_prediction = [90,91,92,93]

In [48]:
L=FINAL(allchampionships,season_to_be_predicted,column_to_predict,column_to_use_as_prediction)

0
1
2
3
4
5
6
7
8
9
10


In [49]:
column_to_predict

'Over1.5/Under1.5'

In [100]:
L

,svm_linear_1,svm_linear_2,svm_linear_3,svm_rbf_1,svm_rbf_2,svm_rbf_3,dtc_1,dtc_2,dtc_3,bagb_1,bagb_2,bagb_3,gradb_1,gradb_2,gradb_3,rdf_1,rdf_2,rdf_3
0,0.828254,0.828254,NaN,0.828254,0.828254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.808254,0.816032,NaN
1,0.763584,0.763584,NaN,0.763584,0.763584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.710012,0.705205,NaN
2,0.771577,0.771577,NaN,0.771577,0.771577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.719494,0.715588,NaN
3,0.753646,0.753646,NaN,0.753646,0.757118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.761632,0.748437,NaN
4,0.777778,0.777778,NaN,0.777778,0.777778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.734127,0.761905,NaN
5,0.722396,0.722396,NaN,0.722396,0.722396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.682465,0.666059,NaN
6,0.800926,0.800926,NaN,0.800926,0.800926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.740741,0.731481,NaN
7,0.686915,0.686915,NaN,0.686915,0.686915,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.647557,0.636049,NaN
8,0.817460,0.817460,NaN,0.817460,0.817460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.793651,0.781746,NaN
9,0.725000,0.725000,NaN,0.725000,0.725000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.643750,0.643750,NaN


## Get The best Methods and Scores for Each League

In [ ]:
for i in range(len(G)):
    
    for i in range(G.iloc[i])
    print ('Best method are', b.mean()[b.mean()==max(b.mean())].index[i], 'with score', b.mean()[b.mean()==max(b.mean())][i])

    print ('Best Methods for' )
    

# Working with cross validation

In [128]:
def find_best_model (X,col_pred,col_train,method):
    n=20
    from sklearn.cross_validation import train_test_split
    tmp=0
    for i in range(n):
        #print i
        X_train, X_test, y_train, y_test = train_test_split(X.iloc[:,column_to_use_as_prediction], X[col_pred], test_size=0.20)
        method.fit(X_train,y_train)
        if method.score(X_test,y_test)>tmp:
            Best=method.fit(X_train,y_train)
            tmp=method.score(X_test,y_test)
    return Best

In [148]:
season_to_be_predicted='2015/2016'
column_to_predict='Over2.5/Under2.5'
column_to_use_as_prediction = [90,91,92,93]

In [141]:
svm_rbf = svm.SVC(kernel='rbf')
all_methods_rbf=[]
for i in range(len(allchampionships)):
    all_methods_rbf.append(MyDf(find_best_model(allchampionships[i].df[allchampionships[i].df.season!=season_to_be_predicted],column_to_predict,column_to_use_as_prediction,svm_rbf))) 
   

In [149]:
ada= AdaBoostClassifier(RandomForestClassifier(max_depth=10, n_estimators=20))
all_methods_ada=[]
for i in range(len(allchampionships)):
    all_methods_ada.append(MyDf(find_best_model(allchampionships[i].df[allchampionships[i].df.season!=season_to_be_predicted],column_to_predict,column_to_use_as_prediction,ada))) 


In [ ]:
svm_linear= svm.SVC(kernel='linear')
all_methods_ada=[]
for i in range(len(allchampionships)):
    all_methods_ada.append(MyDf(find_best_model(allchampionships[i].df[allchampionships[i].df.season!=season_to_be_predicted],column_to_predict,column_to_use_as_prediction,svm_linear))) 


In [150]:
scores=[]
for i in range(len(allchampionships)):
    X=allchampionships[i].df[allchampionships[i].df.season==season_to_be_predicted].iloc[:,column_to_use_as_prediction]
    Y=allchampionships[i].df[allchampionships[i].df.season==season_to_be_predicted][column_to_predict]
    scores.append(all_methods_ada[i].df.score(X,Y))

In [152]:
sum(scores)/len(scores)

0.50195776445776441

# Working with ELO

In [153]:
final_elo=pd.read_csv('../Data/from_sqlite_Data/Realfinal_withELO.csv')

In [160]:
len(final_elo[abs(final_elo['DIFF_ELO'])>100])

11265

In [162]:
len(final_elo)-len(final_elo[abs(final_elo['DIFF_ELO'])>100])

12794

In [163]:
filtered_elo=final_elo[abs(final_elo['DIFF_ELO'])>100]

In [177]:
allchampionships_elo=[]
for i in range(len(final_elo.country_id.unique())):
    allchampionships_elo.append(MyDf(final_elo[final_elo['country_id']==final_elo.country_id.unique()[i]].sort(columns='Date')))
    allchampionships_elo[i].df.reset_index(inplace=True)
    allchampionships_elo[i].df.drop(['index'],axis=1,inplace=True)

/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


In [178]:
pd.DataFrame(allchampionships_elo[0].df.columns)

,0
0,level_0
1,Unnamed: 0
2,Unnamed: 0.1
3,Unnamed: 0.1.1
4,country_id
5,season
6,Date
7,HomeTeam
8,AwayTeam
9,FTHG


In [179]:
allchampionships_elo[0].df.iloc[:,[103,93,92,91]]

,DIFF_ELO,DIFF_av_HP-AP,DIFF_av_TP,DIFF_L5MP
0,-70.517456,0.000000,0.000000,0.0
1,-35.949219,0.000000,0.000000,0.0
2,269.859619,0.000000,0.000000,0.0
3,-121.493042,0.000000,0.000000,0.0
4,-57.016113,0.000000,0.000000,0.0
5,142.763672,0.000000,3.000000,3.0
6,150.361206,0.000000,3.000000,3.0
7,-110.960815,0.000000,0.000000,0.0
8,-282.500000,0.000000,0.000000,0.0
9,312.606934,3.000000,3.000000,3.0


In [ ]:
season_to_be_predicted='2015/2016'
column_to_predict='FTR'
column_to_use_as_prediction = [103,93,92,91]
a=pred_all_meth(allchampionships_elo[0].df, season_to_be_predicted,column_to_predict,column_to_use_as_prediction)

In [ ]:
b=put

# Worling with filtered data and ELO

# Working on sureness

In [826]:
df=allchampionships[0].df
df=df[df['season']==season_to_be_predicted]


In [827]:
for i in range(7,30):
    X_train=df[df.H_ROUND<i].iloc[:,column_to_use_as_prediction]
    Y_train=df[df.H_ROUND<i][column_to_predict]
    X_test=df[df.H_ROUND==i].iloc[:,column_to_use_as_prediction]
    Y_test=df[df.H_ROUND==i][column_to_predict]
    svm_linear = svm.SVC(kernel='linear')
    svm_linear.fit(X_train,Y_train)
    print(svm_linear.decision_function(X_test))
    #print(svm_linear.predict(X_test))
    #print(svm_linear.score(X_test,Y_test))
    #print(Y_test)
    

/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/sklearn/svm/base.py:552: ChangedBehaviorWarning: The decision_function_shape default value will change from 'ovo' to 'ovr' in 0.19. This will change the shape of the decision function returned by SVC.
  "SVC.", ChangedBehaviorWarning)


[[  1.4         -0.47385034  -2.6       ]
 [  1.          -0.15799586  -1.8       ]
 [  4.2          0.78956758  -4.6       ]
 [ 13.4          3.94811239 -12.2       ]
 [  1.          -0.15799586  -1.8       ]]
[[  3.4          1.0000357   -3.        ]
 [ 13.4         -0.99942882 -12.2       ]
 [  1.           1.57137247  -0.6       ]
 [  4.2         -0.14317333  -5.8       ]
 [ -1.4          0.99939312  -0.6       ]]
[[-0.33309724  1.22253381  2.        ]
 [ 0.99988196  0.55547167 -1.        ]
 [ 3.22186809  0.11097929 -4.33333333]
 [-3.22210418  0.99964048  4.33333333]
 [-0.99976391  1.22242595  2.66666667]]
[[ 7.4        -1.15875238 -0.9998161 ]
 [ 5.4        -1.31766032 -1.00001314]
 [ 5.2        -0.90487302 -0.99985551]
 [-2.4         0.4917746  -0.9997767 ]
 [ 7.4        -1.15875238 -0.9998161 ]]
[[ 1.         -0.59100457 -1.        ]
 [ 1.         -0.95391731 -1.        ]
 [ 1.         -1.31882001 -1.        ]
 [ 1.          0.22654408 -1.        ]
 [ 1.         -0.63686615 -1. 

In [142]:
pd.read_csv('../Data/from_sqlite_Data/Team.csv')

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673,KRC Genk,GEN
1,2,9993,675,Beerschot AC,BAC
2,3,10000,15005,SV Zulte-Waregem,ZUL
3,4,9994,2007,Sporting Lokeren,LOK
4,5,9984,1750,KSV Cercle Brugge,CEB
5,6,8635,229,RSC Anderlecht,AND
6,7,9991,674,KAA Gent,GEN
7,8,9998,1747,RAEC Mons,MON
8,9,7947,<null>,FCV Dender EH,DEN
9,10,9985,232,Standard de Liège,STL


In [ ]:
2

# For Real Betting!

In [2]:
premier_2017=pd.read_csv('../Data/from_sqlite_Data/belgium.csv')

In [3]:
final_2=pd.read_csv('../Data/from_sqlite_Data/Realfinal_withELO.csv')

In [4]:
pd.DataFrame(final_2.columns)

,0
0,Unnamed: 0
1,Unnamed: 0.1
2,index
3,Unnamed: 0.1.1
4,country_id
5,season
6,Date
7,HomeTeam
8,AwayTeam
9,FTHG


In [44]:
#column_to_predict='Over2.5/Under2.5'
#column_to_predict='Goal/No_Goal'
column_to_predict='FTR'


column_to_use_as_prediction = [91,92,94,95,103]

In [45]:
final_2.country_id.unique()

array(['Belgium Jupiler League'], dtype=object)

In [46]:
final_2=final_2[final_2.country_id=='Belgium Jupiler League']

In [47]:
X=final_2.iloc[:,column_to_use_as_prediction]
Y=final_2[column_to_predict]


In [48]:
method  = svm.SVC(kernel='rbf')
#method =   svm.SVC(kernel='linear')
#method =  AdaBoostClassifier(RandomForestClassifier(max_depth=10, n_estimators=20))




In [49]:
premier_2017.loc[240]

Unnamed: 0                             240
Date                   2017-01-05 00:00:00
HomeTeam                            Brugge
AwayTeam                           Waregem
FTHG                                     0
FTAG                                     0
FTR                                      D
season                           2016/2017
H_ROUND                                  1
H_TP3                                    0
H_TP5                                    0
H_GS_H                                   0
H_GS_A                                   0
H_GR_H                                   0
H_GR_A                                   0
H_TGS                                    0
H_TGR                                    0
H_AvGS_H                                 0
H_AvGS_A                                 0
H_AvGR_H                                 0
H_AvGR_A                                 0
H_Av_TGS                                 0
H_Av_TGR                                 0
H_Pnt_H    

In [50]:
scores=[]
tmp=0
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.1)
method.fit(X_train,Y_train)

    

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [51]:
TEST=[-7,-1.8,1.86667,1.73333,126]

In [52]:
method.predict(TEST)

/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array(['H'], dtype=object)

In [43]:
#premier_2017.loc[343]

In [190]:
svm_linear = svm.SVC(kernel='linear')
svm_linear.fit(X_train,Y_train)

svm_rbf=svm.SVC(kernel='rbf', gamma=10)
svm_rbf.fit(X_train,Y_train)

rdf = RandomForestClassifier(max_depth=10, n_estimators=20)
rdf.fit(X_train,Y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=20, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [191]:
svm_linear.predict(X_test)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [192]:
svm_rbf.predict(X_test)


array([1, 0, 0, 1, 1, 0, 0, 1, 1, 1])

In [193]:
rdf.predict(X_test)

array([1, 0, 1, 0, 1, 1, 0, 1, 1, 1])

In [137]:
import urllib


In [97]:
premier_2017.HomeTeam.unique()

array(['Burnley', 'Crystal Palace', 'Everton', 'Hull', 'Man City',
       'Middlesbrough', 'Southampton', 'Arsenal', 'Bournemouth', 'Chelsea',
       'Man United', 'Leicester', 'Stoke', 'Swansea', 'Tottenham',
       'Watford', 'West Brom', 'Sunderland', 'West Ham', 'Liverpool'], dtype=object)

In [119]:
import pandas as pd
import io
import requests
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

In [239]:
a=pd.read_csv('../Data/elo/elo_totale')

In [240]:
a['From']=pd.to_datetime(a['From'])
a['To']=pd.to_datetime(a['To'])

In [297]:
lis_home=[]
for i in premier_2017.index:
    tmp=a[a.Club==premier_2017['HomeTeam'][i]]
    lis_home.append(tmp[tmp['To'] >= pd.to_datetime(premier_2017.Date[i])].iloc[0]['Elo'])
    

In [298]:
lis_away=[]
for i in premier_2017.index:
    tmp=a[a.Club==premier_2017['AwayTeam'][i]]
    lis_away.append(tmp[tmp['To'] >= pd.to_datetime(premier_2017.Date[i])].iloc[0]['Elo'])
    
    

In [300]:
premier_2017['ELO_HOME']=lis_home
premier_2017['ELO_AWAY']=lis_away

In [309]:
premier_2017['ELO_DIFF']=premier_2017['ELO_HOME']-premier_2017['ELO_AWAY']